In [86]:
import pandas as pd  

In [87]:
try:
    df = pd.read_csv('data.csv', sep=';', encoding='latin-1')
except Exception as e:
    print(f"Erreur lors du chargement du fichier: {e}")
    # Si le chargement échoue, nous arrêtons ici.
    raise

In [76]:
df = df[df['Sexe'] != 'Sexe - total']

In [77]:
all_Regions = df['Grande région'].unique()
allowed_regions = [region for region in all_Regions if region != 'Suisse']
print(f"Régions identifiées (excluant 'Suisse'): {len(allowed_regions)} - {allowed_regions}")

Régions identifiées (excluant 'Suisse'): 7 - ['Région lémanique', 'Espace Mittelland', 'Nordwestschweiz', 'Zürich', 'Ostschweiz', 'Zentralschweiz', 'Ticino']


In [78]:
df_for_each_region = {}
nombre_df = 0

for region in allowed_regions:
    # Filtrer le DataFrame principal pour la région actuelle et créer une copie propre (.copy())
    df_for_each_region[region] = df[df['Grande région'] == region].copy()
    nombre_df += 1

print(f"\n✅ Opération terminée. {nombre_df} DataFrames créés avec succès.")


✅ Opération terminée. 7 DataFrames créés avec succès.


In [79]:
print("\n--- Aperçu des DataFrames créés ---")
for region, sub_df in df_for_each_region.items():
    # Afficher la taille de chaque nouveau DataFrame
    print(f"**{region}** : {sub_df.shape[0]} lignes, {sub_df.shape[1]} colonnes")
print("\n\n--- Exemples de données dans chaque DataFrame ---\n")
for region, sub_df in df_for_each_region.items():
    print(f"**{region}** : {sub_df.tail(2).to_markdown(index=False)}\n")
    
    # Afficher les 2 premières lignes du premier DataFrame pour démonstration
    # if region == allowed_regions[0]:
    #     print("Voici les 2 premières lignes du DataFrame pour la première région (Zurich) :")
    #     print(sub_df.head(2).to_markdown(index=False))
    #     print("...")


--- Aperçu des DataFrames créés ---
**Région lémanique** : 180 lignes, 10 colonnes
**Espace Mittelland** : 180 lignes, 10 colonnes
**Nordwestschweiz** : 180 lignes, 10 colonnes
**Zürich** : 180 lignes, 10 colonnes
**Ostschweiz** : 180 lignes, 10 colonnes
**Zentralschweiz** : 180 lignes, 10 colonnes
**Ticino** : 180 lignes, 10 colonnes


--- Exemples de données dans chaque DataFrame ---

**Région lémanique** : |   Année | Grande région    | Division économique         | Position professionnelle   | Sexe   |   Médiane |   P10 |   P25 |   P75 |   P90 |
|--------:|:-----------------|:----------------------------|:---------------------------|:-------|----------:|------:|------:|------:|------:|
|    2012 | Région lémanique | > 63 Services d'information | Sans fonction de cadre     | Femmes |       nan |   nan |   nan |   nan |   nan |
|    2012 | Région lémanique | > 63 Services d'information | Sans fonction de cadre     | Hommes |       nan |   nan |   nan |   nan |   nan |

**Espace Mitt

In [80]:
import numpy as np

df_pay_gap_by_region = {}

cols_to_keep = [ 'Année', 'Grande région', 'Division économique',  'Position professionnelle']

for region, df_region in df_for_each_region.items():
   df_cleaned = df_region.replace(['..', ':', ';;', ''], np.nan)
   df_cleaned['Médiane'] = pd.to_numeric(df_cleaned['Médiane'], errors='coerce')
   df_pivot = df_cleaned.pivot_table(
       index=cols_to_keep,
       columns='Sexe',
       values='Médiane',
      aggfunc='first'
   ).reset_index()
   df_pivot.rename(columns={'Hommes': 'Mediane_Homme', 'Femmes': 'Mediane_Femme'}, inplace=True)
   df_pivot['Pay_Gap'] = df_pivot['Mediane_Homme'] - df_pivot['Mediane_Femme']
   final_cols = cols_to_keep + ['Mediane_Homme', 'Mediane_Femme', 'Pay_Gap']
   df_final = df_pivot[final_cols].copy()
   df_pay_gap_by_region[region] = df_final
   print(f"✅ Calcul du Pay Gap pour la région {region} terminé. Taille du DataFrame: {df_final.shape}")

✅ Calcul du Pay Gap pour la région Région lémanique terminé. Taille du DataFrame: (63, 7)
✅ Calcul du Pay Gap pour la région Espace Mittelland terminé. Taille du DataFrame: (72, 7)
✅ Calcul du Pay Gap pour la région Nordwestschweiz terminé. Taille du DataFrame: (60, 7)
✅ Calcul du Pay Gap pour la région Zürich terminé. Taille du DataFrame: (90, 7)
✅ Calcul du Pay Gap pour la région Ostschweiz terminé. Taille du DataFrame: (67, 7)
✅ Calcul du Pay Gap pour la région Zentralschweiz terminé. Taille du DataFrame: (69, 7)
✅ Calcul du Pay Gap pour la région Ticino terminé. Taille du DataFrame: (55, 7)


In [81]:
print("\n--- Aperçu du Résultat (Région de Zurich) ---")
df_zurich_diff = df_pay_gap_by_region['Zürich']
print(df_zurich_diff.tail().to_markdown(index=False))


--- Aperçu du Résultat (Région de Zurich) ---
|   Année | Grande région   | Division économique         | Position professionnelle               |   Mediane_Homme |   Mediane_Femme |   Pay_Gap |
|--------:|:----------------|:----------------------------|:---------------------------------------|----------------:|----------------:|----------:|
|    2022 | Zürich          | > 63 Services d'information | Cadre inférieur                        |           13255 |           12912 |       343 |
|    2022 | Zürich          | > 63 Services d'information | Cadre supérieur et moyen               |           18057 |           18545 |      -488 |
|    2022 | Zürich          | > 63 Services d'information | Position professionnelle - total       |           15374 |           11421 |      3953 |
|    2022 | Zürich          | > 63 Services d'information | Responsable de l'exécution des travaux |           11196 |             nan |       nan |
|    2022 | Zürich          | > 63 Services d'information |

In [82]:
dfs_pay_gap_mean_by_region_year = []

print("Calcul de la moyenne de l'écart salarial (Médiane H-F) par Année et Région...")
for region, df_region in df_pay_gap_by_region.items():
    df_mean = df_region.groupby(['Année', 'Grande région'])['Pay_Gap'].mean().round(2).reset_index()
    df_mean.rename(columns={'Pay_Gap': 'Pay_Gap_Mean'}, inplace=True)
    dfs_pay_gap_mean_by_region_year.append(df_mean)
    print(f"✅ Moyenne calculée pour la région {region}. Taille du DataFrame: {df_mean.shape}")

Calcul de la moyenne de l'écart salarial (Médiane H-F) par Année et Région...
✅ Moyenne calculée pour la région Région lémanique. Taille du DataFrame: (6, 3)
✅ Moyenne calculée pour la région Espace Mittelland. Taille du DataFrame: (6, 3)
✅ Moyenne calculée pour la région Nordwestschweiz. Taille du DataFrame: (6, 3)
✅ Moyenne calculée pour la région Zürich. Taille du DataFrame: (6, 3)
✅ Moyenne calculée pour la région Ostschweiz. Taille du DataFrame: (6, 3)
✅ Moyenne calculée pour la région Zentralschweiz. Taille du DataFrame: (6, 3)
✅ Moyenne calculée pour la région Ticino. Taille du DataFrame: (6, 3)


In [83]:
for df_mean in dfs_pay_gap_mean_by_region_year:
    print(f"{df_mean}\n")

   Année     Grande région  Pay_Gap_Mean
0   2012  Région lémanique       1056.90
1   2014  Région lémanique       1133.70
2   2016  Région lémanique       1264.91
3   2018  Région lémanique       1312.89
4   2020  Région lémanique        526.40
5   2022  Région lémanique        962.10

   Année      Grande région  Pay_Gap_Mean
0   2012  Espace Mittelland       1611.09
1   2014  Espace Mittelland       1580.30
2   2016  Espace Mittelland       1367.20
3   2018  Espace Mittelland       1374.50
4   2020  Espace Mittelland       1530.00
5   2022  Espace Mittelland       1348.18

   Année    Grande région  Pay_Gap_Mean
0   2012  Nordwestschweiz       2089.43
1   2014  Nordwestschweiz       1465.33
2   2016  Nordwestschweiz       1322.17
3   2018  Nordwestschweiz       1339.50
4   2020  Nordwestschweiz       1314.33
5   2022  Nordwestschweiz       1169.17

   Année Grande région  Pay_Gap_Mean
0   2012        Zürich       2766.55
1   2014        Zürich       2136.00
2   2016        Zürich   

In [84]:
df_pay_gap = pd.concat(dfs_pay_gap_mean_by_region_year, ignore_index=True)

df_by_year = {}

annees_uniques = df_pay_gap['Année'].unique()

for annee in annees_uniques:
      df_annee = df_pay_gap[df_pay_gap['Année'] == annee].copy()
      df_by_year[annee] = df_annee

for annee, df_annee in df_by_year.items():
    print(f"{df_annee.to_markdown(index=False)}\n")

|   Année | Grande région     |   Pay_Gap_Mean |
|--------:|:------------------|---------------:|
|    2012 | Région lémanique  |        1056.9  |
|    2012 | Espace Mittelland |        1611.09 |
|    2012 | Nordwestschweiz   |        2089.43 |
|    2012 | Zürich            |        2766.55 |
|    2012 | Ostschweiz        |        2190.2  |
|    2012 | Zentralschweiz    |        1827.2  |
|    2012 | Ticino            |        1695.6  |

|   Année | Grande région     |   Pay_Gap_Mean |
|--------:|:------------------|---------------:|
|    2014 | Région lémanique  |        1133.7  |
|    2014 | Espace Mittelland |        1580.3  |
|    2014 | Nordwestschweiz   |        1465.33 |
|    2014 | Zürich            |        2136    |
|    2014 | Ostschweiz        |        1919.4  |
|    2014 | Zentralschweiz    |        1193.25 |
|    2014 | Ticino            |        1371.4  |

|   Année | Grande région     |   Pay_Gap_Mean |
|--------:|:------------------|---------------:|
|    2016 | Région

In [85]:
df_to_print = {}
for annee, df_annee in df_by_year.items():
    region_pay_gap = df_annee[['Grande région', 'Pay_Gap_Mean']].copy()
    df_to_print[annee] = region_pay_gap.set_index('Grande région')['Pay_Gap_Mean']

for df_annee in df_to_print.values():
    print(f"{df_annee.to_markdown(index=True)}\n")

| Grande région     |   Pay_Gap_Mean |
|:------------------|---------------:|
| Région lémanique  |        1056.9  |
| Espace Mittelland |        1611.09 |
| Nordwestschweiz   |        2089.43 |
| Zürich            |        2766.55 |
| Ostschweiz        |        2190.2  |
| Zentralschweiz    |        1827.2  |
| Ticino            |        1695.6  |

| Grande région     |   Pay_Gap_Mean |
|:------------------|---------------:|
| Région lémanique  |        1133.7  |
| Espace Mittelland |        1580.3  |
| Nordwestschweiz   |        1465.33 |
| Zürich            |        2136    |
| Ostschweiz        |        1919.4  |
| Zentralschweiz    |        1193.25 |
| Ticino            |        1371.4  |

| Grande région     |   Pay_Gap_Mean |
|:------------------|---------------:|
| Région lémanique  |        1264.91 |
| Espace Mittelland |        1367.2  |
| Nordwestschweiz   |        1322.17 |
| Zürich            |        2276.08 |
| Ostschweiz        |        1903    |
| Zentralschweiz    |  